In [28]:
from PIL import Image
from os import listdir
import imageio as iio
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

datapath = './test_dataset/center_crop'
y_path = 'label.txt'
X_path = [f for f in listdir(datapath) if f.endswith('jpg')]
y=[]
X=[]
#target 레이블 저장
with open(f'{datapath}/{y_path}','r') as label_txt:
    y=list(map(lambda x: int(x.split()[0]),label_txt.readlines()))
    le = LabelEncoder()
    y = le.fit_transform(y)
    label_txt.close()
# print(X)
for i in range(len(X_path)):
    img_path = X_path[i]
    img_data=[]
    img = Image.open(f'{datapath}/{img_path}')
    ih, iw = img.size
    pix = img.load()
    # print(iw, ih)
    for py in range(100):
        temp = []
        for px in range(100):
            if py <ih and px < iw :
                pixel = pix[py,px]
                temp.append([pixel[0]/255,pixel[1]/255,pixel[2]/255])
            else :
                temp.append((0,0,0))
        img_data.append(temp)
    X.append(img_data[:])
print(f'{len(X)} datas')

808 datas


In [29]:
X = np.asarray(X)
y= np.asarray(y)
train_images, test_images, train_labels, test_labels = train_test_split(X,y, test_size=0.2,random_state=42)


In [30]:
print('@',train_images, '2',train_labels, '`',test_images, ',.',test_labels)

@ [[[[0.58039216 0.03921569 0.09019608]
   [0.57647059 0.03529412 0.08627451]
   [0.58039216 0.03137255 0.08235294]
   ...
   [0.22745098 0.21568627 0.19607843]
   [0.25882353 0.24705882 0.22745098]
   [0.28627451 0.2745098  0.25490196]]

  [[0.57647059 0.03529412 0.08627451]
   [0.57647059 0.03529412 0.08627451]
   [0.58039216 0.03137255 0.08235294]
   ...
   [0.25882353 0.24705882 0.22745098]
   [0.27058824 0.25882353 0.23921569]
   [0.29411765 0.28235294 0.2627451 ]]

  [[0.57254902 0.03137255 0.08235294]
   [0.57254902 0.03137255 0.08235294]
   [0.58039216 0.03137255 0.08235294]
   ...
   [0.29411765 0.2745098  0.25098039]
   [0.2745098  0.25490196 0.23137255]
   [0.30196078 0.28235294 0.25882353]]

  ...

  [[0.58039216 0.         0.08627451]
   [0.58039216 0.         0.08627451]
   [0.58039216 0.         0.08627451]
   ...
   [0.38823529 0.23921569 0.23529412]
   [0.34509804 0.19607843 0.19215686]
   [0.31372549 0.16470588 0.16078431]]

  [[0.58039216 0.         0.08627451]
   [0

In [43]:
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, KFold
from keras.layers import LeakyReLU
score_acc = make_scorer(accuracy_score)
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')

In [61]:

def nn_func(activation
            # ,optimizer, learning_rate, batch_size, epochs,
            # layers1, layers2, layers3, layers4, dropout_rate,dropout_prob,pooling_size,

            ):
    optimizer = 1
    learning_rate = 0.01
    batch_size = 100
    epochs = 3
    layers1 = 6
    layers2 = 5
    layers3 = 6
    layers4 = 6
    dropout_prob = 0.2
    dropout_rate = 0.3
    pooling_size = 2



    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    layers3 = round(layers3)
    layers4 = round(layers4)
    poolingSize = round(pooling_size)
    def cl_model():
      model = models.Sequential()
      model.add(Conv2D(100, (3, 3), activation=activation, input_shape=(100, 100,3)))
      model.add(MaxPooling2D(pool_size=(poolingSize, poolingSize)))
      model.add(Conv2D(layers1, (3, 3), activation=activation))
      model.add(MaxPooling2D(pool_size=(poolingSize, poolingSize)))
      model.add(Conv2D(layers2, (3, 3), activation=activation))
      model.add(Flatten())
      model.add(Dense(layers3, activation=activation))
      if dropout_prob > 0.5:
        model.add(Dropout(dropout_rate, seed=123))
      model.add(layers.Dense(layers4, activation=activation))
      model.add(Dense(10, activation='softmax'))
      model.compile(optimizer=optimizer,
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
      return model
    # print(cl_model().summary())
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=cl_model, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
    score = cross_val_score(nn, train_images, train_labels, scoring=score_acc, cv=kfold)
    for train_index, test_index in kfold.split(train_images,train_labels):
       print(train_index, test_index)
    model = cl_model()
    y_predict = model.predict(test_images)
    # print('es',es,'nn',nn,'kfold',kfold,'score',score)
    return accuracy_score(test_labels, np.argmax(y_predict,axis=1))




In [51]:
params_nn ={
    'activation':(0, 9),
    # 'optimizer':(0,7),
    # 'learning_rate':(0.01, 1),
    # 'batch_size':(100, 300),
    # 'epochs':(20, 100),
    # 'dropout_prob':(0,1),
    # 'dropout_rate':(0,0.3),
    # 'layers1':(10, 100),
    # 'layers2':(10, 100),
    # 'layers3':(10, 100),
    # 'layers4':(10, 100),
    # 'pooling_size':(2, 5)

}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_func, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... |
-------------------------------------
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
| 1         | 0.006173  | 5.51      |
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
| 2         | 0.0       | 1.522     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
| 3         | 0.1111    | 3.925     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
| 4         | 0.4938    | 6.923     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
| 5         | 0.142     | 2.658     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
| 6         | 0.0       | 1.342     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
| 7         | 0.1111    | 0.2023    |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
| 8         | 0.0       | 3.782     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
| 9         | 0.01852   | 2.148     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
| 10        | 0.5679    | 3.039     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
| 11        | 0.0       | 8.916     |
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
| 12        | 0.1049    | 2.14      |
6/6 ━━━━━━━━

In [62]:
res = nn_func(1)
print(res)


[  0   3   6   8  10  12  13  15  16  17  18  20  21  23  25  28  29  30
  32  33  35  36  37  40  41  43  46  47  49  50  51  52  54  55  56  57
  59  60  63  64  68  70  71  72  73  74  75  76  77  78  79  82  90  91
  92  93  94  95  96  97  98 105 106 107 109 110 111 112 115 116 119 121
 122 123 124 126 127 129 130 132 135 136 137 139 140 141 144 147 152 153
 155 156 158 159 161 163 164 166 167 168 169 170 171 172 173 174 175 178
 179 181 182 183 184 185 186 187 189 190 191 192 193 194 196 197 198 199
 200 201 203 206 207 208 209 211 214 215 217 219 220 221 223 224 225 226
 227 228 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 249
 253 254 256 257 258 259 262 263 264 265 266 267 268 269 270 271 273 274
 277 278 282 283 285 286 290 291 295 296 297 298 300 301 303 304 305 306
 307 308 310 311 312 313 314 315 316 317 318 320 321 322 324 326 327 330
 332 333 334 335 337 338 339 342 343 344 346 347 348 349 350 351 352 353
 354 355 356 357 358 359 360 363 365 367 372 373 37